# 10 Provider Comparison (Experiment)
Read first: `examples/11_provider_comparison.py`

Run the same prompt across multiple providers, measure time, and see responses.


## Select providers
Set `PROVIDER_KEYS` to any of: `sonnet`, `opus`, `gpt`, `gpt-codex`.
The helper will resolve paths and ensure env vars are set.


In [ ]:
# Choose which providers to compare
PROVIDER_KEYS = ["sonnet", "gpt"]

In [ ]:
# Resolve provider paths and foundation
import os
import sys
from pathlib import Path

ROOT = Path.cwd()
if ROOT.name == "notebooks":
    ROOT = ROOT.parent
sys.path.insert(0, str(ROOT))

from utils.providers import PROVIDERS  # noqa: E402
from utils.providers import select_provider  # noqa: E402

FOUNDATION_PATH, _, _ = select_provider(PROVIDER_KEYS[0], foundation="minimal", prompt_for_env=False)
PROVIDER_PATHS = [PROVIDERS[k] for k in PROVIDER_KEYS]
print("Comparing providers:")
for k in PROVIDER_KEYS:
    print(f"  - {k}: {PROVIDERS[k].name}")
# Ensure env vars for all selected
for k in PROVIDER_KEYS:
    env = "OPENAI_API_KEY" if "openai" in PROVIDERS[k].name.lower() else "ANTHROPIC_API_KEY"
    if not os.getenv(env):
        try:
            import getpass

            os.environ[env] = getpass.getpass(f"Enter {env}: ")
            print(f"Set {env} for this kernel session.")
        except Exception:
            print(f"{env} missing; set it before running.")
    else:
        print(f"{env} already set.")

In [ ]:
# Parameters (edit me)
PROMPT = "Explain recursion in programming with a simple Python example."

In [ ]:
import time
from dataclasses import dataclass
from pathlib import Path

from amplifier_foundation import load_bundle


@dataclass
class ProviderResult:
    provider_name: str
    model_name: str
    duration: float
    response: str


async def test_provider(provider_path: Path, prompt: str) -> ProviderResult:
    foundation = await load_bundle(str(FOUNDATION_PATH))
    provider = await load_bundle(str(provider_path))
    model_name = "unknown"
    if provider.providers:
        model_name = provider.providers[0].get("config", {}).get("default_model", "unknown")
    composed = foundation.compose(provider)
    prepared = await composed.prepare()
    session = await prepared.create_session()
    start = time.time()
    async with session:
        response = await session.execute(prompt)
    duration = time.time() - start
    return ProviderResult(provider.name, model_name, duration, response)


async def compare():
    results = []
    for path in PROVIDER_PATHS:
        print(f"\nTesting provider: {path.name}")
        res = await test_provider(path, PROMPT)
        results.append(res)
    return results


results = await compare()
print("\nSummary:")
for r in results:
    preview = r.response[:1000] + ("..." if len(r.response) > 1000 else "")
    print(f"- {r.provider_name} ({r.model_name}) took {r.duration:.2f}s")
    print(f"  Preview: {preview}")

### Try this
- Add/remove entries in `PROVIDER_KEYS` to compare different models.
- Change `PROMPT` to a task from your domain and compare outputs.
- Set `OPENAI_API_KEY` to include OpenAI providers.
